Some popular financial news websites that can be considered for web scraping:
 - Yahoo Finance
 - Seeking Alpha
 - Market warch
 - Motley Fool
 - Market Watch
 - Investing

Our scraped data should only include headline, label, date, company ticker

## Market Watch Scraper (3 scrapers)
- Market Watch, Motely Fool, Benzinga

### First level: Company ticker

### Second Level: Date

### Third Level: Headline, URL

Scrape news from the top 90 companies from the file resutlseval.xlsx

In [ ]:
import pandas as pd
df_eval = pd.read_excel("/content/drive/MyDrive/GTMSA/CSE 6242/FinSight Generator Project/resultseval.xlsx",header=None)
df_eval[:91]

,0,1,2,3,4
0,ticker,total,positive,negative,net positive percentage
1,ida,907,57,0,6.284454
2,ew,1134,81,11,6.17284
3,ftnt,1476,95,5,6.097561
4,anss,1057,63,0,5.960265
...,...,...,...,...,...
86,fls,2105,28,9,0.902613
87,dpz,666,6,0,0.900901
88,rezi,1461,21,8,0.889802
89,lcii,1138,10,0,0.878735


In [ ]:
df_eval.rename(columns={0: "ticker"}, inplace=True)
us_active_symbols = list(df_eval['ticker'][1:])

In [ ]:
import re
import csv
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from pytz import timezone
import pandas as pd

In [ ]:
def matches_pattern(url):
    # Define the pattern to match
    pattern = r'^https://www\.marketwatch\.com/investing/stock/[^/]+?mod=mw_quote_news$'
    return bool(re.match(pattern, url))

In [ ]:
def parse_date(date_string):
    date_string = date_string.strip().replace("Published: ", "").replace('First ','').replace(" at ", " ").replace(' ET', '').replace(".", "").replace("\n", "")
    try:
        # parse the date string with the full month name
        return datetime.strptime(date_string, "%B %d, %Y %I:%M %p")
    except ValueError:
        try:
            # parse with the abbreviated month name
            return datetime.strptime(date_string, "%b %d, %Y %I:%M %p")
        except:
            return ''

market watch scraper

In [ ]:
def mW_getElements(url):
    # Find the title, content, and date tags
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_tag = ''
    try:
      title_tag = soup.find('h1', class_='article__headline').text.strip()
    except AttributeError:
      title_tag = ''
    date_tag = ''
    try:
      date_string = soup.find('time', class_='timestamp timestamp--pub').text.strip()
      date_tag = parse_date(date_string)
    except AttributeError:
      try:
        date_string = soup.find('time', class_='timestamp timestamp--update').text.strip()
        date_tag = parse_date(date_string)
      except AttributeError:
        date_tag = ''
    # try:
    #   content_tag = soup.find('div', class_='article__content').text.strip().replace("\t", "").replace("\n", "")
    #   content_tag = re.sub(r'\s+', ' ', content_tag)
    # except:
    #   content_tag = ''
    url_tag = url
    cols = [title_tag, date_tag, url_tag]
    col_names = ['Headline', 'Date', 'URL']
    return dict(zip(col_names, cols))

Motely Fool scraper

In [ ]:
def fool_getElements(url):
  # Find the title, content, and date tags
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  title_tag = ''
  try:
    title_tag = soup.find('h1', class_='font-medium text-gray-1100 leading-42 md:text-h1').text.strip()
  except:
    title_tag = ''
  date_tag2 = ''
  try:
    date_string = soup.find('div', class_='text-lg font-medium text-gray-800 my-24px').text.strip()
    date_tag = date_string.split("\n–\n")[1].strip().replace(" at ", " ").replace("Updated ", "")
    date_tag2 = datetime.strptime(date_tag, "%b %d, %Y %I:%M%p")
  except IndexError:
    pass
  # try:
  #   content_tag = soup.find('div', class_='tailwind-article-body').text.strip().replace("%\xa0", "").replace("\n", "")
  # except:
  #   content_tag = ''
  url_tag = url
  cols = [title_tag, date_tag2, url_tag]
  col_names = ['Headline', 'Date', 'URL']
  return dict(zip(col_names, cols))

Collect and aggregate the articles for the same publicly traded companies

In [ ]:
def web_scraper(ticker: str) -> pd.DataFrame:

  url_mp = 'https://www.marketwatch.com/investing/stock/{}?mod=search_symbol'.format(ticker)
  response_mp = requests.get(url_mp)
  soup = BeautifulSoup(response_mp.text)
  anchor_tags = soup.find_all(lambda tag: tag.name == 'a' and
                                    tag.get('class') == ['link'])

  # Collect the links and count them
  links = []
  for tag in anchor_tags:
      link = tag.get('href')
      if link and link not in links:
          links.append(link)
  links = [l for l in links if l.endswith("quote_news")]

  #exclude home page'https://www.marketwatch.com/investing/stock/{}?mod=mw_quote_news'
  page = 'https://www.marketwatch.com/investing/stock/{}?mod=mw_quote_news'.format(ticker.lower())
  elements = [mW_getElements(url) for url in links if 'marketwatch.com/' in url and url!=page and '/livecoverage/' not in url and matches_pattern(url)== False]
  elements2 = [fool_getElements(url) for url in links if 'fool.com/' in url]
  result = elements+ elements2
  df = pd.DataFrame(result)

  return df

In [ ]:
def result(ticker):
  table = pd.DataFrame(columns=['Headline', 'Date', 'URL','Company'])
  df = web_scraper(ticker)
  df['Company'] = ticker
  table = pd.concat([table, df], ignore_index=True)
  return table

In [ ]:
for i in range(len(us_active_symbols[50:91])):
  ticker = us_active_symbols[50+i]
  final = result(ticker)
  final.to_csv('US_Traded{}.csv'.format(ticker))

<ipython-input-8-16c1ab273cc8>:22: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.DataFrame(result)
<ipython-input-8-16c1ab273cc8>:22: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.DataFrame(result)
<ipython-input-8-16c1ab273cc8>:22: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df = pd.DataFrame(result)
<ipython-input-8-16c1ab273cc8>:22: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Ser

In [ ]:
import re

def benz_scraper(url):
  # Find the title, content, and date tags
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  title_tag = ''
  try:
    title_tag = soup.find('h1', class_='sc-fKVqWL bmZDfz mb-4 layout-title').text.strip()
  except:
    title_tag = ''
  date_tag2 = ''
  try:
    pattern = r'([A-Za-z]+\s\d{1,2},\s\d{4}\s\d{1,2}:\d{2}\s(?:AM|PM))'
    date_string = soup.find('div', class_='article-date-wrap').text.strip()
    match = re.search(pattern, date_string)
    if match:
      date_tag2 = match.group(1)
      date_tag2 = datetime.strptime(date_tag2, "%B %d, %Y %I:%M %p")
    else:
      date_tag2 = ''
  except IndexError:
    pass
  url_tag = url
  cols = [title_tag, date_tag2, url_tag]
  col_names = ['Headline', 'Date', 'URL']
  return dict(zip(col_names, cols))

In [ ]:
def benzinga_collect(ticker):
  url_mp = 'https://www.marketwatch.com/investing/stock/{}?mod=search_symbol'.format(ticker)
  response_mp = requests.get(url_mp)
  soup = BeautifulSoup(response_mp.text)
  anchor_tags = soup.find_all('a', class_='link')
  links = []
  ends_str = 'utm_campaign=partner_feed&utm_source=MarketWatch&utm_medium=partner_feed&utm_content=site&mod=mw_quote_news'

  for tag in anchor_tags:
      link = tag.get('href')
      if link and link not in links:
          links.append(link)

  links = [l for l in links if l.endswith(ends_str)]
  articles = [benz_scraper(url) for url in links if 'benzinga.com/' in url]
  df = pd.DataFrame(articles)
  return df

In [ ]:
def result_benz(ticker):
  table = pd.DataFrame(columns=['Headline', 'Date', 'URL','Company'])
  df = benzinga_collect(ticker)
  df['Company'] = ticker
  table = pd.concat([table, df], ignore_index=True)
  return table

Novanta only have articles from benzinga.com

In [ ]:
result_benz('novt')

,Headline,Date,URL,Company
0,"Roku Has 26% Of Cash With SVB Financial, Depos...",2023-03-11 14:27:00,https://www.benzinga.com/news/23/03/31309113/r...,novt
1,Novanta's Return On Capital Employed Insights,2023-03-07 09:46:00,https://www.benzinga.com/news/earnings/23/03/3...,novt
2,Novanta General Counsel Awarded $510K Worth of...,2023-03-01 10:01:00,https://www.benzinga.com/news/23/03/31126729/n...,novt
3,"Earnings Scheduled For March 1, 2023",2023-03-01 06:59:00,https://www.benzinga.com/news/earnings/23/03/3...,novt
4,Novanta's Return On Capital Employed Overview,2022-12-06 09:46:00,https://www.benzinga.com/news/earnings/22/12/2...,novt
5,Novanta's Return on Invested Capital Insights,2022-09-02 10:23:00,https://www.benzinga.com/news/earnings/22/09/2...,novt
6,Novanta: Q2 Earnings Insights,2022-08-09 07:32:00,https://www.benzinga.com/news/earnings/22/08/2...,novt
7,"HCA Healthcare, Boston Beer Company And Some O...",2022-07-25 06:15:00,https://www.benzinga.com/news/22/07/28183264/h...,novt


In [ ]:
result_benz('novt').to_csv('novt.csv')

For the company SpareBank 1 SMN, there is no related news on MarketWatch.

In [ ]:
benzinga_collect('sji')

""


In [ ]:

for i in range()